# Linking DEC Metadata with J2SR Metrics

## Step 1: Import Packages

In [1]:
import pandas as pd
import numpy as np
import pycountry
import re 

pd.options.display.max_colwidth = 100

## Step 2: Import DEC Data and Create Unique Country-Year ID

In [2]:
dec_meta = pd.read_csv('Raw_Data/dec-evaluations-data.csv', index_col = "Unique_ID", encoding = 'latin')

In [3]:
dec_meta.head()

,Abstract,Ancillary_Data,Bibliographic_Type,ContentType,Contract_Grant_Number,Credit,Date_Resource_Created,Description,Descriptors_Topical,Descriptors_Geographic,...,Publication_Date_Freeform,Related_Doc_Links,Report_Number,Series_Title,Title,Title_Translated,URI,USAID_Geography,USAID_Project_Number,Digital_Object_Identifier
Unique_ID,,,,,,,,,,,,,,,,,,,,,
PD-AAB-495-E1,NaN,Evaluation period: 22 Mar 1972-12 Jul 1976,Final Evaluation Report,Documents,NaN,NaN,7/1/1977 0:00,NaN,Management education ~|~_©_~|~ Graduate education ~|~_©_~|~ Limited data,Central America,...,13-Jul-77,NaN,NaN,NaN,LOAN COMPLETION REVIEW AND REPORT,NaN,https://dec.usaid.gov/dec/content/Detail.aspx?ctID=ODVhZjk4NWQtM2YyMi00YjRmLTkxNjktZTcxMjM2NDBmY...,Reg Office Cent America & Panama (ROCAP),5960044,NaN
PD-AAP-604,NaN,Evaluation period: 25 Jul 1972-10 Apr 1979,Final Evaluation Report,Documents,NaN,NaN,6/1/1979 0:00,NaN,Low cost housing ~|~_©_~|~ Minimum shelter housing ~|~_©_~|~ Housing ~|~_©_~|~ Earthquakes ~|~_©...,Peru,...,6-Jun-79,NaN,NaN,NaN,Urban reconstruction -- earthquake and flood zones,NaN,https://dec.usaid.gov/dec/content/Detail.aspx?ctID=ODVhZjk4NWQtM2YyMi00YjRmLTkxNjktZTcxMjM2NDBmY...,Peru,5270101,NaN
PD-AAB-088-A1,NaN,NaN,Special Evaluation,Documents,AID/TA-C-1469,NaN,2/1/1979 0:00,NaN,Road construction ~|~_©_~|~ Roads ~|~_©_~|~ Rural areas,Honduras,...,1-Feb-79,NaN,NaN,NaN,TRIALS SELECTION CRITERIA AND EVALUATION DESIGNS FOR THE HONDURAS RURAL TRIALS PROJECT,NaN,https://dec.usaid.gov/dec/content/Detail.aspx?ctID=ODVhZjk4NWQtM2YyMi00YjRmLTkxNjktZTcxMjM2NDBmY...,Honduras,5220137,NaN
PD-AAA-880-D1,NaN,Evaluation period: 22 Jan 1979-9 Feb 1979,Final Evaluation Report,Documents,AID/LAC-C-1313,NaN,2/1/1979 0:00,NaN,Health surveys ~|~_©_~|~ Demographic research ~|~_©_~|~ MIGRATION ~|~_©_~|~ Samples ~|~_©_~|~ Su...,El Salvador,...,9-Feb-79,NaN,NaN,NaN,EVALUATION OF MULTIPURPOSE HOUSEHOLD SURVEY (EL SALVADOR) (1/22-2/9/79),NaN,https://dec.usaid.gov/dec/content/Detail.aspx?ctID=ODVhZjk4NWQtM2YyMi00YjRmLTkxNjktZTcxMjM2NDBmY...,El Salvador,5190176,NaN
PD-AAT-461,NaN,Evaluation period: 1 Jan1972-31 Dec1976,Special Evaluation,Documents,AID/pha/C-1100,NaN,7/1/1977 0:00,NaN,Condoms ~|~_©_~|~ Sterilization (birth control) ~|~_©_~|~ Oral contraceptives ~|~_©_~|~ Family p...,Thailand,...,26-Jul-77,NaN,NaN,NaN,REPORT OF THE SECOND EVALUATION OF THE NATIONAL PLANNING PROGRAM IN THAILAND (6/21/77-7/26/77),NaN,https://dec.usaid.gov/dec/content/Detail.aspx?ctID=ODVhZjk4NWQtM2YyMi00YjRmLTkxNjktZTcxMjM2NDBmY...,Thailand,4930283,NaN


In [4]:
dec_meta['New_Thesaurus_Terms'].isna().sum()

5097

In [5]:
dec_meta.shape

(12411, 32)

### Restrict to Recent DEC Final Evaluations

In [6]:
dec_meta['Date_Form'] = pd.to_datetime(dec_meta['Date_Resource_Created'], infer_datetime_format = True)

In [7]:
dec_meta['year'] = pd.DatetimeIndex(dec_meta['Date_Form']).year
dec_recent = dec_meta[dec_meta['year'] > 2009]
dec_recent.shape

(2320, 34)

In [8]:
dec_recent.loc[:,'year'] = dec_recent['year'].astype(str)
dec_recent.loc[:,'year'] = dec_recent['year'].str[:-2]
dec_recent['year']

/home/aaron/anaconda3/envs/dec_j2sr/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Unique_ID
PA-00K-SQC    2015
PA-00K-SGR    2015
PA-00K-SBZ    2015
PA-00K-SBV    2015
PA-00K-SB1    2014
              ... 
PA-00S-RX6    2017
PA-00S-RV5    2016
PA-00S-RSS    2016
PA-00S-RRZ    2017
PA-00S-RRS    2016
Name: year, Length: 2320, dtype: object

In [9]:
# Restrict reports to "Final Evaluation Report" type
dec_recent = dec_recent[dec_recent['Bibliographic_Type'] == 'Final Evaluation Report']
dec_recent.shape

(1149, 34)

### Match Country Codes to Geographic Descriptors using pycountry

In [10]:
def do_fuzzy_search(country):
    try:
        result = pycountry.countries.search_fuzzy(country)
    except Exception:
        return np.nan
    else:
        return result[0].alpha_3

In [11]:
iso_map = {country: do_fuzzy_search(country) for country in dec_recent["Descriptors_Geographic"].unique()}
dec_recent.loc[:,"country_code"] = dec_recent["Descriptors_Geographic"].map(iso_map)

In [12]:
# Create a country_year identifier to match with J2SR Data
dec_recent.loc[:,'country_year'] = dec_recent['country_code'] + "_" + dec_recent['year']
dec_recent['country_year']

Unique_ID
PA-00K-SQC    LBR_2015
PA-00K-SGR    UGA_2015
PA-00K-S9J    IDN_2015
PA-00K-S72         NaN
PA-00K-S5M    UGA_2015
                ...   
PA-00S-S59         NaN
PA-00S-RXV    UGA_2015
PA-00S-RX6         NaN
PA-00S-RV5         NaN
PA-00S-RRZ    MDG_2017
Name: country_year, Length: 1149, dtype: object

## Step 3: Import J2SR Data and Create Unique Country-Year ID

In [13]:
j2sr_data = pd.read_csv('Raw_Data/J2SR_FY21_clean_long.csv')
j2sr_data["country_code"] = j2sr_data["state_country_name"].map(iso_map)
j2sr_data['country_year'] = j2sr_data['country_code'] + "_" + j2sr_data['year'].astype(str)
j2sr_data

,country_id,state_country_name,income_group,region,subregion,year,metric,value,country_code,country_year
0,484,Mexico,UMIC,Latin America and the Caribbean,Central America,2013,Liberal Democracy,0.562997,MEX,MEX_2013
1,484,Mexico,UMIC,Latin America and the Caribbean,Central America,2014,Liberal Democracy,0.569807,MEX,MEX_2014
2,484,Mexico,UMIC,Latin America and the Caribbean,Central America,2015,Liberal Democracy,0.481271,MEX,MEX_2015
3,484,Mexico,UMIC,Latin America and the Caribbean,Central America,2016,Liberal Democracy,0.530079,MEX,MEX_2016
4,484,Mexico,UMIC,Latin America and the Caribbean,Central America,2017,Liberal Democracy,0.500568,MEX,MEX_2017
...,...,...,...,...,...,...,...,...,...,...
14167,704,Vietnam,LMIC,Asia,East Asia,2013,Export Sophistication,0.527244,VNM,VNM_2013
14168,887,Yemen,LIC,Middle East and North Africa,Middle East,2013,Export Sophistication,0.286973,YEM,YEM_2013
14169,710,South Africa,UMIC,Sub-Saharan Africa,Southern Africa,2013,Export Sophistication,0.547118,ZAF,ZAF_2013
14170,894,Zambia,LMIC,Sub-Saharan Africa,Southern Africa,2013,Export Sophistication,0.469175,ZMB,ZMB_2013


In [14]:
# Drop values missing the country_year identifier
j2sr_roadmap = j2sr_data.dropna(subset=['country_year'])
j2sr_roadmap['country_year']

0        MEX_2013
1        MEX_2014
2        MEX_2015
3        MEX_2016
4        MEX_2017
           ...   
14167    VNM_2013
14168    YEM_2013
14169    ZAF_2013
14170    ZMB_2013
14171    ZWE_2013
Name: country_year, Length: 6782, dtype: object

## Step 4: Pivot J2SR Data to Generate Columns per Indicator

In [15]:
j2sr_pivot = j2sr_roadmap.pivot_table(index = 'country_year', columns = 'metric', values = 'value')
j2sr_pivot['iso'] = j2sr_pivot.index.str[:3]
j2sr_pivot['year'] = j2sr_pivot.index.str[4:]
j2sr_pivot

metric,Business & Investment Environment,Child Health,Civil Society & Media Effectiveness,Economic Gender Gap,Education Quality,Environmental Policy,Export Sophistication,GDP Per Capita (PPP),Government Effectiveness,Information & Communication Technology (ICT) Adoption,Liberal Democracy,Open Government,Poverty Rate ($5/Day),Safety & Security,Social Group Equality,Tax System Effectiveness,Trade Freedom,iso,year
country_year,,,,,,,,,,,,,,,,,,,
AFG_2013,0.148340,0.460997,0.757576,NaN,NaN,0.111111,NaN,0.215275,0.232532,NaN,0.257662,NaN,NaN,0.145163,0.344033,0.064815,NaN,AFG,2013
AFG_2014,0.132643,0.484766,0.745037,NaN,NaN,NaN,NaN,0.213937,0.243862,NaN,0.291714,0.303145,NaN,0.071384,0.323324,0.055556,NaN,AFG,2014
AFG_2015,0.123575,0.508202,0.749216,NaN,NaN,0.222222,NaN,0.210658,0.245646,NaN,0.280363,NaN,NaN,0.107041,0.323324,0.074074,NaN,AFG,2015
AFG_2016,0.123021,0.531075,0.715778,NaN,NaN,NaN,NaN,0.209569,0.266091,NaN,0.256527,0.253979,NaN,0.059889,0.288118,0.092593,NaN,AFG,2016
AFG_2017,0.116907,0.553617,0.726228,NaN,NaN,0.222222,NaN,0.209733,0.243182,NaN,0.254257,0.198620,NaN,0.046728,0.293813,0.101852,0.474453,AFG,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWE_2015,0.162010,0.519634,0.435737,0.720182,NaN,0.111111,0.370259,0.268855,0.280587,NaN,0.220204,NaN,NaN,0.539609,0.375356,0.500000,NaN,ZWE,2015
ZWE_2016,0.172253,0.525270,0.384535,0.715257,NaN,NaN,0.347524,0.267258,0.280485,0.244128,0.192963,0.100788,NaN,0.518202,0.413409,0.407407,NaN,ZWE,2016
ZWE_2017,0.175587,0.522055,0.428422,0.743037,NaN,0.222222,0.348357,0.273545,0.274057,0.255277,0.238365,0.109028,0.193206,0.559778,0.569764,0.435185,0.531022,ZWE,2017


## Step 5: Calculate Commitment and Capacity Scores from Normalized Roadmap Indicators

#### Min-Max Scaling
URL: https://selfreliance.usaid.gov/docs/FY_2020_USAID_Journey_to_Self-Reliance_Country_Roadmap_Methodology_Guide.pdf

USAID’s country roadmaps use a min-max scaling technique to normalize all data onto a common 0.0 to
1.0 scale to facilitate visualization, comparison across metrics, and calculation of the Commitment and
Capacity indices. A country scoring 0.0 on a given metric indicates that the country recorded the least
favorable outcome globally in the raw dataset, and a country scoring 1.0 indicates that the country
recorded the most favorable outcome globally in the raw dataset. All other countries receive scores
within the 0.0-1.0 range based on where they fall between the worst and best outcomes globally,
preserving the source organization’s data distribution.

While USAID Roadmaps are only produced for low- and middle-income countries, all countries globally, 
including high-income countries, are used to establish the range of possible outcomes for each metric. 

The period of performance used to determine the range of observed outcomes is 2010 to the latest data 
available on July 1 2019 (including values that have been “carried forward” from 2006-2009 into this 
date range; see “Temporal Coverage” and “Handling Missing Data” sections below for more details)

In [19]:
# Min-Max scaling

# Conducted before for this dataset

#### Methodology from USAID J2SR FY2020 Metrics Guide
URL: https://selfreliance.usaid.gov/docs/FY_2020_USAID_Journey_to_Self-Reliance_Country_Roadmap_Methodology_Guide.pdf

Overall “Commitment” and “Capacity” composite scores are calculated using the arithmetic mean of all
available scaled components for each country. 

The Commitment Index comprises seven underlying metrics, each receiving an equal weight 
(i.e. one-seventh weighting, if all sub-components are present after imputation). 

- Liberal Democracy Index
- Open Government
- Social Group Equality
- Economic Gender Gap
- Business Environment
- Trade Freedom
- Biodiversity and Habitat Protections

The Capacity Index comprises ten underlying metrics, each receiving an equal
weighting of one-tenth in aggregation, if all sub-components are present after imputation. 

- Government Effectiveness
- Tax System Effectiveness
- Safety and Security
- Civil Society and Media Effectiveness
- Poverty Rate
- Education Quality
- Child Health
- GDP per Capita in PPP
- ICT Adoption
- Export Sophistication

If dimension components (i.e. individual metrics) of either index are missing after imputation, Commitment and
Capacity scores are still generated using an arithmetic mean of all available components, but only when
at least six of ten Capacity metrics are present and four of seven Commitment metrics are present.


In [16]:
comm_vars = ['Liberal Democracy',
             'Open Government',
             'Social Group Equality',
             'Economic Gender Gap',
            'Business & Investment Environment',
            'Trade Freedom',
            'Environmental Policy']

In [17]:
j2sr_pivot['comm_missing'] = j2sr_pivot[comm_vars].isnull().sum(axis=1)
j2sr_pivot['comm_missing']

country_year
AFG_2013    3
AFG_2014    3
AFG_2015    3
AFG_2016    3
AFG_2017    1
           ..
ZWE_2015    2
ZWE_2016    2
ZWE_2017    0
ZWE_2018    1
ZWE_2019    2
Name: comm_missing, Length: 574, dtype: int64

In [18]:
# Create function that checks for sufficient components in Commitment and computes arithmetic mean per row
# At least 4 of 7 commitment variables must be present, so at most 3 can be missing
def comm_calc(data, comm_vars):
    if (data['comm_missing'] <= 3):
        comm_score = np.nanmean(data[comm_vars], 0)
        return comm_score

In [19]:
for ind in j2sr_pivot.index:
    j2sr_pivot.loc[ind,'comm_score'] = comm_calc(j2sr_pivot.loc[ind], comm_vars)

In [20]:
cap_vars = ['Child Health',
       'Civil Society & Media Effectiveness', 
       'Education Quality',
       'Export Sophistication',
       'GDP Per Capita (PPP)', 
       'Government Effectiveness',
       'Information & Communication Technology (ICT) Adoption',
       'Poverty Rate ($5/Day)',
       'Safety & Security',
       'Tax System Effectiveness',
]

j2sr_pivot['cap_missing'] = j2sr_pivot[cap_vars].isnull().sum(axis=1)
j2sr_pivot['cap_missing']

country_year
AFG_2013    4
AFG_2014    4
AFG_2015    4
AFG_2016    4
AFG_2017    4
           ..
ZWE_2015    3
ZWE_2016    2
ZWE_2017    1
ZWE_2018    2
ZWE_2019    8
Name: cap_missing, Length: 574, dtype: int64

In [21]:
# Create function that checks for sufficient components in Commitment and computes arithmetic mean per row
# At least 6 of 10 capacity variables must be present, so at most 4 can be missing

def cap_calc(data, cap_vars):
    if (data['cap_missing'] <= 4):
        cap_score = np.nanmean(data[cap_vars], 0)
        return cap_score

In [22]:
for ind in j2sr_pivot.index:
    j2sr_pivot.loc[ind,'cap_score'] = cap_calc(j2sr_pivot.loc[ind], cap_vars)
    
j2sr_pivot

metric,Business & Investment Environment,Child Health,Civil Society & Media Effectiveness,Economic Gender Gap,Education Quality,Environmental Policy,Export Sophistication,GDP Per Capita (PPP),Government Effectiveness,Information & Communication Technology (ICT) Adoption,...,Safety & Security,Social Group Equality,Tax System Effectiveness,Trade Freedom,iso,year,comm_missing,comm_score,cap_missing,cap_score
country_year,,,,,,,,,,,,,,,,,,,,,
AFG_2013,0.148340,0.460997,0.757576,NaN,NaN,0.111111,NaN,0.215275,0.232532,NaN,...,0.145163,0.344033,0.064815,NaN,AFG,2013,3,0.215286,4,0.312726
AFG_2014,0.132643,0.484766,0.745037,NaN,NaN,NaN,NaN,0.213937,0.243862,NaN,...,0.071384,0.323324,0.055556,NaN,AFG,2014,3,0.262706,4,0.302424
AFG_2015,0.123575,0.508202,0.749216,NaN,NaN,0.222222,NaN,0.210658,0.245646,NaN,...,0.107041,0.323324,0.074074,NaN,AFG,2015,3,0.237371,4,0.315806
AFG_2016,0.123021,0.531075,0.715778,NaN,NaN,NaN,NaN,0.209569,0.266091,NaN,...,0.059889,0.288118,0.092593,NaN,AFG,2016,3,0.230411,4,0.312499
AFG_2017,0.116907,0.553617,0.726228,NaN,NaN,0.222222,NaN,0.209733,0.243182,NaN,...,0.046728,0.293813,0.101852,0.474453,AFG,2017,1,0.260045,4,0.313557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWE_2015,0.162010,0.519634,0.435737,0.720182,NaN,0.111111,0.370259,0.268855,0.280587,NaN,...,0.539609,0.375356,0.500000,NaN,ZWE,2015,2,0.317773,3,0.416383
ZWE_2016,0.172253,0.525270,0.384535,0.715257,NaN,NaN,0.347524,0.267258,0.280485,0.244128,...,0.518202,0.413409,0.407407,NaN,ZWE,2016,2,0.318934,2,0.371851
ZWE_2017,0.175587,0.522055,0.428422,0.743037,NaN,0.222222,0.348357,0.273545,0.274057,0.255277,...,0.559778,0.569764,0.435185,0.531022,ZWE,2017,0,0.369861,1,0.365543


## Step 6: J2SR Nearest Neighbor Analysis

In [23]:
# Use spatial distance functions from scipy package to calculate the distance matrix for all country-year combinations

from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

dist_matrix = pd.DataFrame(
    squareform(pdist(j2sr_pivot.loc[:,['comm_score','cap_score']])),
    columns = j2sr_pivot.index,
    index = j2sr_pivot.index
)

dist_matrix

country_year,AFG_2013,AFG_2014,AFG_2015,AFG_2016,AFG_2017,AFG_2018,AFG_2019,AGO_2013,AGO_2014,AGO_2015,...,ZMB_2017,ZMB_2018,ZMB_2019,ZWE_2013,ZWE_2014,ZWE_2015,ZWE_2016,ZWE_2017,ZWE_2018,ZWE_2019
country_year,,,,,,,,,,,,,,,,,,,,,
AFG_2013,0.000000,0.048526,0.022298,0.015126,0.044766,0.085983,NaN,0.132994,0.139335,0.135166,...,0.261604,0.334811,NaN,0.158566,0.147438,0.145767,0.119325,0.163349,0.191805,NaN
AFG_2014,0.048526,0.000000,0.028653,0.033830,0.011447,0.046518,NaN,0.114699,0.113827,0.121349,...,0.222811,0.296035,NaN,0.135956,0.129508,0.126566,0.089341,0.124363,0.159460,NaN
AFG_2015,0.022298,0.028653,0.000000,0.007705,0.022786,0.063776,NaN,0.116116,0.120349,0.119851,...,0.239969,0.313240,NaN,0.140621,0.130873,0.128764,0.098963,0.141518,0.171390,NaN
AFG_2016,0.015126,0.033830,0.007705,0.000000,0.029653,0.071344,NaN,0.123043,0.127744,0.126389,...,0.247665,0.320940,NaN,0.147815,0.137741,0.135735,0.106578,0.149197,0.179029,NaN
AFG_2017,0.044766,0.011447,0.022786,0.029653,0.000000,0.043000,NaN,0.105779,0.106429,0.111749,...,0.220160,0.293477,NaN,0.128079,0.120653,0.117923,0.082862,0.121499,0.154288,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWE_2015,0.145767,0.126566,0.128764,0.135735,0.117923,0.087181,NaN,0.012780,0.021494,0.017517,...,0.141426,0.208878,NaN,0.015716,0.004532,0.000000,0.044547,0.072787,0.063534,NaN
ZWE_2016,0.119325,0.089341,0.098963,0.106578,0.082862,0.045169,NaN,0.037220,0.025984,0.049577,...,0.144590,0.217098,NaN,0.048912,0.048750,0.044547,0.000000,0.051316,0.072495,NaN
ZWE_2017,0.163349,0.124363,0.141518,0.149197,0.121499,0.078512,NaN,0.074374,0.053442,0.086992,...,0.098661,0.171978,NaN,0.064413,0.077030,0.072787,0.051316,0.000000,0.045266,NaN


In [24]:
# If the column name contains the iso code, change the value in the cell to nan
# Eliminates countries from matching with themselves

for row in range(len(dist_matrix)):
    for col in range(len(dist_matrix)):
        if (j2sr_pivot['iso'][row] in dist_matrix.columns[col]):
            dist_matrix.iloc[row,col] = np.nan
            
dist_matrix

country_year,AFG_2013,AFG_2014,AFG_2015,AFG_2016,AFG_2017,AFG_2018,AFG_2019,AGO_2013,AGO_2014,AGO_2015,...,ZMB_2017,ZMB_2018,ZMB_2019,ZWE_2013,ZWE_2014,ZWE_2015,ZWE_2016,ZWE_2017,ZWE_2018,ZWE_2019
country_year,,,,,,,,,,,,,,,,,,,,,
AFG_2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.132994,0.139335,0.135166,...,0.261604,0.334811,NaN,0.158566,0.147438,0.145767,0.119325,0.163349,0.191805,NaN
AFG_2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.114699,0.113827,0.121349,...,0.222811,0.296035,NaN,0.135956,0.129508,0.126566,0.089341,0.124363,0.159460,NaN
AFG_2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.116116,0.120349,0.119851,...,0.239969,0.313240,NaN,0.140621,0.130873,0.128764,0.098963,0.141518,0.171390,NaN
AFG_2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.123043,0.127744,0.126389,...,0.247665,0.320940,NaN,0.147815,0.137741,0.135735,0.106578,0.149197,0.179029,NaN
AFG_2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.105779,0.106429,0.111749,...,0.220160,0.293477,NaN,0.128079,0.120653,0.117923,0.082862,0.121499,0.154288,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWE_2015,0.145767,0.126566,0.128764,0.135735,0.117923,0.087181,NaN,0.012780,0.021494,0.017517,...,0.141426,0.208878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZWE_2016,0.119325,0.089341,0.098963,0.106578,0.082862,0.045169,NaN,0.037220,0.025984,0.049577,...,0.144590,0.217098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZWE_2017,0.163349,0.124363,0.141518,0.149197,0.121499,0.078512,NaN,0.074374,0.053442,0.086992,...,0.098661,0.171978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Set n as the number of matches to make between country_year combinations in the distance matrix
# Results in n columns with nearest country-year matches

n = 5

for row in range(len(dist_matrix)):
    j2sr_index = dist_matrix.index[row]
    n_smallest = np.argsort(np.array(dist_matrix.iloc[row,]))[0:n]
    for neighbor in range(n):
        col_name = str(neighbor) + "_near"
        j2sr_pivot.loc[j2sr_index, col_name] = dist_matrix.iloc[row,n_smallest].index.values[neighbor]

j2sr_pivot

metric,Business & Investment Environment,Child Health,Civil Society & Media Effectiveness,Economic Gender Gap,Education Quality,Environmental Policy,Export Sophistication,GDP Per Capita (PPP),Government Effectiveness,Information & Communication Technology (ICT) Adoption,...,year,comm_missing,comm_score,cap_missing,cap_score,0_near,1_near,2_near,3_near,4_near
country_year,,,,,,,,,,,,,,,,,,,,,
AFG_2013,0.148340,0.460997,0.757576,NaN,NaN,0.111111,NaN,0.215275,0.232532,NaN,...,2013,3,0.215286,4,0.312726,SDN_2017,ETH_2017,ETH_2016,YEM_2014,SDN_2018
AFG_2014,0.132643,0.484766,0.745037,NaN,NaN,NaN,NaN,0.213937,0.243862,NaN,...,2014,3,0.262706,4,0.302424,ETH_2018,ETH_2017,ETH_2015,ETH_2016,ETH_2014
AFG_2015,0.123575,0.508202,0.749216,NaN,NaN,0.222222,NaN,0.210658,0.245646,NaN,...,2015,3,0.237371,4,0.315806,SDN_2017,ETH_2017,ETH_2018,ETH_2016,ETH_2014
AFG_2016,0.123021,0.531075,0.715778,NaN,NaN,NaN,NaN,0.209569,0.266091,NaN,...,2016,3,0.230411,4,0.312499,SDN_2017,ETH_2017,ETH_2016,ETH_2018,ETH_2014
AFG_2017,0.116907,0.553617,0.726228,NaN,NaN,0.222222,NaN,0.209733,0.243182,NaN,...,2017,1,0.260045,4,0.313557,ETH_2018,ETH_2014,ETH_2015,ETH_2017,ETH_2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWE_2015,0.162010,0.519634,0.435737,0.720182,NaN,0.111111,0.370259,0.268855,0.280587,NaN,...,2015,2,0.317773,3,0.416383,MRT_2013,MRT_2017,BGD_2017,AGO_2013,AGO_2016
ZWE_2016,0.172253,0.525270,0.384535,0.715257,NaN,NaN,0.347524,0.267258,0.280485,0.244128,...,2016,2,0.318934,2,0.371851,AGO_2017,AGO_2014,AGO_2018,PAK_2018,HTI_2018
ZWE_2017,0.175587,0.522055,0.428422,0.743037,NaN,0.222222,0.348357,0.273545,0.274057,0.255277,...,2017,0,0.369861,1,0.365543,AGO_2018,SLE_2013,AGO_2017,MDG_2013,SLE_2015


In [35]:
j2sr_pivot.loc[:,"0_near":]

metric,0_near,1_near,2_near,3_near,4_near
country_year,,,,,
AFG_2013,SDN_2017,ETH_2017,ETH_2016,YEM_2014,SDN_2018
AFG_2014,ETH_2018,ETH_2017,ETH_2015,ETH_2016,ETH_2014
AFG_2015,SDN_2017,ETH_2017,ETH_2018,ETH_2016,ETH_2014
AFG_2016,SDN_2017,ETH_2017,ETH_2016,ETH_2018,ETH_2014
AFG_2017,ETH_2018,ETH_2014,ETH_2015,ETH_2017,ETH_2016
...,...,...,...,...,...
ZWE_2015,MRT_2013,MRT_2017,BGD_2017,AGO_2013,AGO_2016
ZWE_2016,AGO_2017,AGO_2014,AGO_2018,PAK_2018,HTI_2018
ZWE_2017,AGO_2018,SLE_2013,AGO_2017,MDG_2013,SLE_2015


## Step 7: Link DEC and J2SR Data

In [26]:
# Link DEC reports with J2SR data using a left-side merge on the country-year identifier

dec_j2sr = pd.merge(dec_recent, j2sr_pivot, how='left', left_on = 'country_year', right_index=True)
dec_j2sr.head()

,Abstract,Ancillary_Data,Bibliographic_Type,ContentType,Contract_Grant_Number,Credit,Date_Resource_Created,Description,Descriptors_Topical,Descriptors_Geographic,...,year_y,comm_missing,comm_score,cap_missing,cap_score,0_near,1_near,2_near,3_near,4_near
Unique_ID,,,,,,,,,,,,,,,,,,,,,
PA-00K-SQC,NaN,Submitted as: Final evaluation of the Liberia grants and solicitation mechanism (LGSM) project,Final Evaluation Report,Documents,NaN,NaN,3/1/2015 0:00,NaN,Quality of care ~|~_©_~|~ Orphans and vulnerable children (OVC) ~|~_©_~|~ Human capacity develop...,Liberia,...,2015,2.0,0.532831,3.0,0.416186,BFA_2017,MOZ_2015,MOZ_2013,TZA_2014,MWI_2018
PA-00K-SGR,NaN,"""Submitted to: USAID/Uganda"" ~|~_©_~|~ Submitted as: SUNRISE-OVC final evaluation ~|~_©_~|~ Foot...",Final Evaluation Report,Documents,AID-OAA-A-14-00061,NaN,9/1/2015 0:00,NaN,Orphans and vulnerable children (OVC) ~|~_©_~|~ Health service utilization ~|~_©_~|~ Access to s...,Uganda,...,2015,2.0,0.491794,3.0,0.405585,TJK_2016,MDG_2017,MLI_2015,TZA_2018,MOZ_2017
PA-00K-S9J,NaN,Project title: Cross sectoral strategies for climate change and disaster risk reduction (CADRE),Final Evaluation Report,Documents,AID-497-A-11-00015,NaN,6/1/2015 0:00,NaN,Climate change ~|~_©_~|~ Disaster relief ~|~_©_~|~ Vulnerable groups ~|~_©_~|~ Rural areas ~|~_©...,Indonesia,...,2015,2.0,0.530838,2.0,0.546312,NPL_2015,AZE_2018,NPL_2017,NPL_2018,KGZ_2016
PA-00K-S72,NaN,Submitted as: East Africa trade hub (EATH) final evaluation report ~|~_©_~|~ Evaluated task orde...,Final Evaluation Report,Documents,AID-OAA-M-13-00012,NaN,8/1/2015 0:00,NaN,Trade promotion ~|~_©_~|~ International trade ~|~_©_~|~ Nontariff barriers ~|~_©_~|~ Economic ef...,East Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PA-00K-S5M,NaN,"Submitted as: Performance evaluation of STRIDES for family health program, USAID/Uganda (2009-20...",Final Evaluation Report,Documents,AID-617-BC-14-00008 ~|~_©_~|~ AID-617-E-13-00011,NaN,1/1/2015 0:00,NaN,Family health care ~|~_©_~|~ Health service utilization ~|~_©_~|~ Quality of care ~|~_©_~|~ Heal...,Uganda,...,2015,2.0,0.491794,3.0,0.405585,TJK_2016,MDG_2017,MLI_2015,TZA_2018,MOZ_2017


In [27]:
dec_j2sr.to_csv('Working_Data/dec_j2sr_data.csv')